In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api = os.getenv("GROQ_API_KEY")

llm = ChatGroq(model="llama-3.3-70b-versatile")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000018AD6ED1940>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000018AD6ED2660>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [10]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

In [7]:
embedding = OllamaEmbeddings(model="embeddinggemma:300m")

In [33]:
from bs4 import SoupStrainer
loader = WebBaseLoader(web_path=("https://en.wikipedia.org/wiki/World_War_II"))
docs = loader.load()

In [16]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,chunk_overlap=300)
splits=text_splitter.split_documents(docs)
vector_store=Chroma.from_documents(documents=splits,embedding=embedding)
retriever=vector_store.as_retriever()

In [18]:
system_prompt = (
    "You are an assistant for question answering tasks. "
    "Use the following piece of retrieved context to answer the question. "
    "If you do not know the answer, say that you do not know. "
    "Use three sentences maximum. Keep the answer concise.\n{context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [19]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [25]:
from IPython.display import Markdown

In [43]:
response=rag_chain.invoke({"input":"how many soldiers were in the red army?"})
display(Markdown(response['answer']))

The total number of soldiers in the Red Army during World War II is not explicitly stated in the provided context. However, it is mentioned that the Soviet Union lost around 8.7 million military personnel during the war.

In [68]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_id,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [69]:
response = conversational_rag_chain.invoke(
    {"input":"who led germany during ww2"},
    config={"configurable":{"session_id":"abc123"}}
)['answer']
display(Markdown(response))

Adolf Hitler led Germany during World War II. He was the main Axis leader of Germany, along with Hirohito of Japan and Benito Mussolini of Italy.

In [70]:
response = conversational_rag_chain.invoke(
    {"input":"which countries were invaded by him?"},
    config={"configurable":{"session_id":"abc123"}}
)['answer']
display(Markdown(response))

Adolf Hitler and Nazi Germany invaded several countries during World War II, including Poland, France, Belgium, Netherlands, Denmark, Norway, Greece, Yugoslavia, and the Soviet Union. They also launched attacks on the United Kingdom and other countries.

In [71]:
response = conversational_rag_chain.invoke(
    {"input":"whom did i ask about?"},
    config={"configurable":{"session_id":"abc123"}}
)['answer']
display(Markdown(response))

You asked about Adolf Hitler, the leader of Germany during World War II.

In [73]:
get_session_id("abc123")

InMemoryChatMessageHistory(messages=[HumanMessage(content='who led germany during ww2', additional_kwargs={}, response_metadata={}), AIMessage(content='Adolf Hitler led Germany during World War II. He was the main Axis leader of Germany, along with Hirohito of Japan and Benito Mussolini of Italy.', additional_kwargs={}, response_metadata={}), HumanMessage(content='which countries were invaded by him?', additional_kwargs={}, response_metadata={}), AIMessage(content='Adolf Hitler and Nazi Germany invaded several countries during World War II, including Poland, France, Belgium, Netherlands, Denmark, Norway, Greece, Yugoslavia, and the Soviet Union. They also launched attacks on the United Kingdom and other countries.', additional_kwargs={}, response_metadata={}), HumanMessage(content='whom did i ask about?', additional_kwargs={}, response_metadata={}), AIMessage(content='You asked about Adolf Hitler, the leader of Germany during World War II.', additional_kwargs={}, response_metadata={})]

[HumanMessage(content='who led the soviet union', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The leader of the Soviet Union during World War II was Joseph Stalin.', additional_kwargs={}, response_metadata={})]